<a href="https://colab.research.google.com/github/tlsehdgns1999/tlsehdgns1999/blob/main/airline_delay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### 필요 모듈 불러오기

import pandas as pd
import numpy as np
import os
import gc

In [3]:
### 베이스 라인 코드를 빌려옴(실행속도 향상)

def csv_to_parquet(csv_path, save_name):
     df = pd.read_csv(csv_path)
     df.to_parquet(f'./{save_name}.parquet')
     del df
     gc.collect()
     print(save_name, 'Done.')

train_path = 'data/train.csv'
test_path = 'data/test.csv'

csv_to_parquet(train_path, 'train')
csv_to_parquet(test_path, 'test')

train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
sample_submission = pd.read_csv('data/sample_submission.csv', index_col = 0)

train Done.
test Done.


In [4]:
### Month 와 Date를 한 가지의 days 컬럼으로 합치는 과정 (train, test 적용)

def days(x) :
    month_days = {1:0, 2:31, 3:60, 4:91, 5:121, 6:152, 7:182, 8:213, 9:244, 10:274, 11:305, 12:335}
    return month_days[x]

df_list = [train, test]
for i in range(len(df_list)) :
    df_list[i].loc[:, 'Day'] = train['Month'].apply(lambda x : days(x))
    df_list[i]['Day'] = df_list[i]['Day'] + df_list[i]['Day_of_Month']

train = train.astype({'Day': object})
test = test.astype({'Day': object})

### 'Month', 'Date' 컬럼 제거

train_new = train.drop([train.columns[1], train.columns[2]], axis = 1)
test_new = test.drop([test.columns[1], test.columns[2]], axis = 1)

In [5]:
### 'Carrier_ID(DOT)' 는 'Airline'에 일대일 대응되는 지표이다. 
### 일대일 대응 되는 지표가 있으면 대체 하는 과정 

carrier_code_mapping = train_new.groupby('Airline')['Carrier_ID(DOT)'].apply(lambda x: x.mode().iloc[0])
train_new['Carrier_ID(DOT)'] = train_new['Carrier_ID(DOT)'].fillna(train_new['Airline'].map(carrier_code_mapping))

In [6]:
airline_code_mapping = train_new.groupby('Carrier_ID(DOT)')['Airline'].apply(lambda x: x.mode().iloc[0])
train_new['Airline'] = train_new['Airline'].fillna(train_new['Carrier_ID(DOT)'].map(airline_code_mapping))

In [7]:
### 대응 후 남은 'Carrier_ID(DOT)', 'Airline' 의 결측치 

train_new.isnull().sum()

ID                               0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                      11883
Carrier_Code(IATA)          108990
Carrier_ID(DOT)              11883
Tail_Number                      0
Delay                       744999
Day                              0
dtype: int64

In [8]:
### train set 이므로 남는 결측치는 전부 제거

train_new = train_new.dropna(subset = ['Carrier_ID(DOT)'], how='any', axis = 0)

In [9]:
train_new.isnull().sum()

ID                               0
Estimated_Departure_Time    107726
Estimated_Arrival_Time      107789
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                107739
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           107782
Distance                         0
Airline                          0
Carrier_Code(IATA)          107717
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       736101
Day                              0
dtype: int64

In [10]:
### test도 같은 방식으로 진행, 하지만 없어지는 행이 없어야 하기 때문에 최빈값으로 대체하는 작업 필요

carrier_code_mapping2 = test_new.groupby('Airline')['Carrier_ID(DOT)'].apply(lambda x: x.mode().iloc[0])
test_new['Carrier_ID(DOT)'] = test_new['Carrier_ID(DOT)'].fillna(test_new['Airline'].map(carrier_code_mapping2))

In [11]:
airline_code_mapping2 = test_new.groupby('Carrier_ID(DOT)')['Airline'].apply(lambda x: x.mode().iloc[0])
test_new['Airline'] = test_new['Airline'].fillna(test_new['Carrier_ID(DOT)'].map(airline_code_mapping2))

In [12]:
test_new.isnull().sum()

ID                               0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                      11543
Carrier_Code(IATA)          108993
Carrier_ID(DOT)              11543
Tail_Number                      0
Day                              0
dtype: int64

In [13]:
from scipy.stats import mode

### 결측치 대체할 최빈값 계산
mode_value = test_new['Carrier_ID(DOT)'].mode().iloc[0]
test_new['Carrier_ID(DOT)'] = test_new['Carrier_ID(DOT)'].fillna(mode_value)

mode_value = test_new['Airline'].mode().iloc[0]
test_new['Airline'] = test_new['Airline'].fillna(mode_value)

In [14]:
test_new.isnull().sum()

ID                               0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                          0
Carrier_Code(IATA)          108993
Carrier_ID(DOT)                  0
Tail_Number                      0
Day                              0
dtype: int64

In [15]:
### 남는 열 제거(중복사항)

col_drop = ['Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train_new = train_new.drop(col_drop, axis=1)
test_new = test_new.drop(col_drop, axis=1)

In [16]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 988117 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        988117 non-null  object 
 1   Estimated_Departure_Time  880391 non-null  float64
 2   Estimated_Arrival_Time    880328 non-null  float64
 3   Origin_Airport_ID         988117 non-null  int64  
 4   Destination_Airport_ID    988117 non-null  int64  
 5   Distance                  988117 non-null  float64
 6   Carrier_ID(DOT)           988117 non-null  float64
 7   Tail_Number               988117 non-null  object 
 8   Delay                     252016 non-null  object 
 9   Day                       988117 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 82.9+ MB


In [17]:
test_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Estimated_Departure_Time  891016 non-null   float64
 2   Estimated_Arrival_Time    890952 non-null   float64
 3   Origin_Airport_ID         1000000 non-null  int64  
 4   Destination_Airport_ID    1000000 non-null  int64  
 5   Distance                  1000000 non-null  float64
 6   Carrier_ID(DOT)           1000000 non-null  float64
 7   Tail_Number               1000000 non-null  object 
 8   Day                       1000000 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 68.7+ MB


In [18]:
train_new.head()

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay,Day
0,TRAIN_000000,NaN,NaN,13851,12191,419.0,19393.0,N7858A,None,106
1,TRAIN_000001,740.0,1024.0,13930,14869,1250.0,20304.0,N125SY,None,228
2,TRAIN_000002,1610.0,1805.0,11057,12953,544.0,19805.0,N103US,None,250
3,TRAIN_000003,905.0,1735.0,12892,11618,2454.0,19977.0,N595UA,None,192
4,TRAIN_000004,900.0,1019.0,14771,10157,250.0,20304.0,N161SY,None,11


In [19]:
### 이 작업은 대강의 분 차이를 이용해 출발 또는 도착의 시간을 예상해 결측치를 채워넣는 방법이다.

### 먼저 'HHMM' 를 분으로 변환

def con_to_minutes(x) :
    if np.isnan(x) :
        return np.nan
    else:
        x = int(x)
        x = str(x)
        if len(x) > 2 :
            # 시간과 분으로 나눔
            hours = int(x[:-2])
            mins = int(x[-2:])
        else :
            hours = 0
            mins = int(x[-2:])
        return hours * 60 + mins

### 'Estimated_Departure_Time' 열의 값을 분으로 변환

train_new['Estimated_Departure_Time'] = train_new['Estimated_Departure_Time'].apply(con_to_minutes)

### 'Estimated_Arrival_Time' 열의 값을 분으로 변환

train_new['Estimated_Arrival_Time'] = train_new['Estimated_Arrival_Time'].apply(con_to_minutes)

In [20]:
train_new = train_new.dropna(subset = ['Estimated_Arrival_Time', 'Estimated_Departure_Time'], how = 'all', axis = 0)

In [21]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 976567 entries, 1 to 999999
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        976567 non-null  object 
 1   Estimated_Departure_Time  880391 non-null  float64
 2   Estimated_Arrival_Time    880328 non-null  float64
 3   Origin_Airport_ID         976567 non-null  int64  
 4   Destination_Airport_ID    976567 non-null  int64  
 5   Distance                  976567 non-null  float64
 6   Carrier_ID(DOT)           976567 non-null  float64
 7   Tail_Number               976567 non-null  object 
 8   Delay                     249021 non-null  object 
 9   Day                       976567 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 82.0+ MB


In [22]:
### Origin_Airport_ID와 Destination_Airport_ID를 키로, 평균 비행시간을 값으로 갖는 디렉토리 생성

from collections import defaultdict

time_flying = defaultdict(int)
time_number = defaultdict(int)

cond = ~train_new['Estimated_Arrival_Time'].isnull() & ~train_new['Estimated_Departure_Time'].isnull()

for _, row in train_new[cond].iterrows():
    origin, dest = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    flying_time = (row['Estimated_Arrival_Time'] - row['Estimated_Departure_Time']) % 1440
    time_flying[(origin, dest)] += flying_time
    time_number[(origin, dest)] += 1

for key in time_flying:
    time_flying[key] /= time_number[key]

In [23]:
### 결측값 채우기

for idx, row in train_new[train_new['Origin_Airport_ID'].isnull() | train_new['Destination_Airport_ID'].isnull()].iterrows():
    origin, dest = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    if origin in time_flying and not pd.isnull(origin):
        train_new.at[idx, 'Origin_Airport_ID'] = origin
    if dest in time_flying and not pd.isnull(dest):
        train_new.loc[idx, 'Destination_Airport_ID'] = dest
        train_new.loc[idx, 'Flying_Time'] = time_flying[(origin, dest)]

In [24]:
### Estimated_Arrival_Time - Estimated_Departure_Time 을 계산해 같은 출발, 도착지를 갖는 행의 결측치를 채움

mask_dep = train_new['Estimated_Departure_Time'].isnull()
mask_arr = train_new['Estimated_Arrival_Time'].isnull()

train_new.loc[mask_dep, 'Estimated_Departure_Time'] = train_new.loc[mask_dep].apply(lambda row: (row['Estimated_Arrival_Time'] - time_flying[(row['Origin_Airport_ID'], row['Destination_Airport_ID'])]) % 1440, axis=1)
train_new.loc[mask_arr, 'Estimated_Arrival_Time'] = train_new.loc[mask_arr].apply(lambda row: (row['Estimated_Departure_Time'] + time_flying[(row['Origin_Airport_ID'], row['Destination_Airport_ID'])]) % 1440, axis=1)


In [25]:
### test의 결측치를 최빈값으로 대체

mode_dep = test_new['Estimated_Departure_Time'].mode().iloc[0]
mode_arr = test_new['Estimated_Arrival_Time'].mode().iloc[0]

test_new['Estimated_Departure_Time'].fillna(mode_dep, inplace=True)
test_new['Estimated_Arrival_Time'].fillna(mode_arr, inplace=True)

In [26]:
### Dealy 의 NaN 값 제거 (라벨링 된 데이터로만 추론 진행)

train_new = train_new.dropna()

In [27]:
### Delayed, Not_Delayed 를 0, 1 로 카테고리 화, astype을 쓰지 않은 이유는 변환을 명확히 하기 위함

train_new['Delay'] = train_new['Delay'].replace('Delayed', 1)
train_new['Delay'] = train_new['Delay'].replace('Not_Delayed', 0)

In [28]:
### test셋에 비해 train셋의 양이 현저히 적어졌으므로 validation셋을 생성

from sklearn.model_selection import train_test_split


X = train_new.drop(columns=['ID', 'Delay'], axis = 1)
y = train_new['Delay']
test_val = test_new.drop(columns=['ID'])
X_train, X_valid , y_train, y_valid = train_test_split(X, y, test_size = 0.2, stratify = y)

In [29]:
### xgb 학습을 위해 질적 변수를 양적 변수로 변환함

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

q_col = ['Tail_Number', 'Day']

for i in q_col:
    le = LabelEncoder()
    le=le.fit(X_train[i])
    X_train[i]=le.transform(X_train[i])

    for label in np.unique(X_valid[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    X_valid[i]=le.transform(X_valid[i])

    for label in np.unique(test_new[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_new[i]=le.transform(test_new[i])
print('Done.')

Done.


In [30]:
### 모델링

model = XGBClassifier(
                      booster = 'gbtree',
                      scale_pos_weight=5,
                      learning_rate=0.01,  
                      colsample_bytree = 0.7,
                      subsample = 0.5,
                      max_delta_step = 3,
                      reg_lambda = 2,
                     objective='binary:logistic',
                      n_estimators=818, 
                      max_depth=8,
                     )

In [31]:
%%time
eval_set = [(X_valid, y_valid)]
eval_metric = ["logloss"]
model.fit(X_train, y_train,early_stopping_rounds=50, eval_metric=eval_metric, eval_set = eval_set)

[0]	validation_0-logloss:0.69277


C:\Users\cjswo\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\cjswo\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-logloss:0.69235
[2]	validation_0-logloss:0.69188
[3]	validation_0-logloss:0.69150
[4]	validation_0-logloss:0.69104
[5]	validation_0-logloss:0.69065
[6]	validation_0-logloss:0.69022
[7]	validation_0-logloss:0.68981
[8]	validation_0-logloss:0.68944
[9]	validation_0-logloss:0.68904
[10]	validation_0-logloss:0.68859
[11]	validation_0-logloss:0.68822
[12]	validation_0-logloss:0.68782
[13]	validation_0-logloss:0.68740
[14]	validation_0-logloss:0.68706
[15]	validation_0-logloss:0.68671
[16]	validation_0-logloss:0.68663
[17]	validation_0-logloss:0.68626
[18]	validation_0-logloss:0.68595
[19]	validation_0-logloss:0.68569
[20]	validation_0-logloss:0.68562
[21]	validation_0-logloss:0.68532
[22]	validation_0-logloss:0.68522
[23]	validation_0-logloss:0.68493
[24]	validation_0-logloss:0.68459
[25]	validation_0-logloss:0.68426
[26]	validation_0-logloss:0.68396
[27]	validation_0-logloss:0.68392
[28]	validation_0-logloss:0.68360
[29]	validation_0-logloss:0.68328
[30]	validation_0-loglo

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=3,
              max_depth=8, max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=818, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [32]:
predictions = model.predict_proba(X_valid)
predictions

array([[0.56815386, 0.43184614],
       [0.4873038 , 0.5126962 ],
       [0.3070565 , 0.6929435 ],
       ...,
       [0.67282385, 0.32717615],
       [0.6016617 , 0.39833832],
       [0.4602949 , 0.5397051 ]], dtype=float32)

In [37]:
predictions = model.predict_proba(test_new)
predictions

ValueError: Feature shape mismatch, expected: 8, got 9

In [34]:
sample_submission = pd.read_csv('data/sample_submission.csv', index_col = 0)

In [35]:
submission = pd.DataFrame(data=predictions, columns=sample_submission.columns, index=sample_submission.index)

ValueError: Shape of passed values is (49805, 2), indices imply (1000000, 2)

In [36]:
submission.to_csv('data/sample_submission.csv', index=True)

NameError: name 'submission' is not defined